In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%cd drive/MyDrive

# 建模流程

在這一小節中將會進行 YOLOv7 模型的訓練、預測並評估其表現結果。還沒看過 4.3-directions.md 的同學要先去看一下喔！

第一步，先將 yolov7 github clone 至自己的資料路徑，並進入該資料夾中。


In [ ]:
! git clone https://github.com/WongKinYiu/yolov7

In [ ]:
%cd yolov7

首先來看一下檔案內部結構

* cfg：存放用於 training、deploy 或 baseline 的模型架構 yaml 檔案

* data：存放用於訓練過程中的資料檔案 (coco.yaml) 及超參數設置檔 (hyp.scratch.xxx.yaml)

* deploy/triton-inference-server：將 YOLOv7 模型部署至 NVIDIA 開源的 Inference Server

* inference/images：存放可進行測試的圖片

* models：存放模型架構的定義以及一些 NMS operation 操作

* scripts：可藉由 get_coco.sh 檔案下載 COCO dataset

* tools：存放 YOLOv7 在各種應用上的範例

* utils：存放 dataloader、loss、activation function 等檔案，用於訓練或驗證階段

* detect.py：用於進行推理運算預測

* export.py：用於轉換模型

* hubconf.py：用於支援 Pytorch Hub

* test.py：用於計算評估指標

* requirement.txt：訓練和測試過程中所需要的套件

* train.py：用於訓練模型

* train_aux.py：加入 auxiliary head 用於訓練模型


## 資料前處理 (Data Preprocessing)

### 載入資料集
在進行訓練之前，要先準備資料集，這邊使用 [Kaggle Car Object Detection](https://www.kaggle.com/datasets/sshikamaru/car-object-detection/code)，總共有 1001 張 training 175 張 test data。

我們使用 kaggle api 進行下載並解壓縮放入 data_file 資料夾中。

In [ ]:
! pip install -q kaggle

In [ ]:
# 這邊選擇剛下載的 kaggle.json

from google.colab import files
files.upload()

In [ ]:
!rm -r ~/.kaggle
!mkdir ~/.kaggle
!mv ./kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets list

In [ ]:
! mkdir data_file

In [ ]:
data_save = os.path.join(os.getcwd(), 'data_file/')

In [ ]:
! kaggle datasets download -d sshikamaru/car-object-detection -p {data_save} --unzip

### 檢查資料集

在訓練模型之前，我們需要先對資料集進行檢查，以下分為幾個部分：

* 資料集格式

  - 檢查資料集格式是否為 YOLO 格式，若是其他格式要先將其進行轉換。

* 資料集劃分

  - 通常會將資料集劃分為訓練集 (training set)、驗證集 (validation set)、測試集 (test set)
    - 訓練集 (training set)：用於訓練模型參數
    - 驗證集 (validation set)：用於檢驗模型的訓練狀況，作為調整超參數的依據
    - 測試集 (test set)：評估模型的表現結果

* 資料集存放路徑

  - YOLO 在訓練時，會從使用者建立的 train.txt 和 valid.txt 裡的路徑去尋找 training data 及 validation data (產生 train.txt、valid.txt 的步驟會在後續進行)。

  - 需要注意的是，在 train.txt、valid.txt 中的資料路徑只需要填入資料集的圖片路徑，訓練時除了尋找資料集圖片，也會去尋找在同一個資料夾中同檔名的標記檔案 (txt 檔)。因此要將圖片和標記檔放在同一個資料夾中。


### 觀察資料

下載完 Kaggle Car Object Detection 資料集後，會看到以下檔案

* training_images：用於訓練的資料集

* testing_images：用於測試的資料集

* train_solution_bounding_boxes (1).csv：訓練資料的 label

* sample_submission.csv：用於上傳測試 label 的檔案

首先來讀取訓練資料的路徑以及訓練資料的 label csv 檔，然後將物件框的資訊取出來，畫圖看看會呈現什麼樣子吧！

In [ ]:
import os
import shutil
import yaml
import pandas as pd
import cv2
import matplotlib.pyplot as plt

In [ ]:
images_path = os.path.join(os.getcwd(), 'data_file/data/training_images')
data_path = os.path.join(os.getcwd(), 'data_file/data/train_solution_bounding_boxes (1).csv')

In [ ]:
df_data = pd.read_csv(data_path)
df_data

In [ ]:
image = df_data['image'][0]
xmin = int(df_data['xmin'][0])
ymin = int(df_data['ymin'][0])
xmax = int(df_data['xmax'][0])
ymax = int(df_data['ymax'][0])

In [ ]:
img = cv2.imread(os.path.join(images_path, image))
cv2.rectangle(img, (xmin, ymin), (xmax, ymax), (255, 0, 255), 3)
img_h, img_w, img_c = img.shape

img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
plt.imshow(img)

### 資料格式轉換

由於資料集是使用 VOC xml 格式 (xmin, ymin, xmax, ymax)，因此需要先將其轉換成 YOLO 格式。

在 4.1、4.2 節中有介紹格式轉換及實作，我們可以將程式碼包裝成一個函式進行使用。

先來試試看轉換上一張圖片的格式吧！

In [ ]:
def convert_to_yolo(img_w, img_h, xmin, ymin, xmax, ymax):
  dw = 1./img_w
  dh = 1./img_h
  x_center = ((xmin + xmax) /2) * dw
  y_center = ((ymin + ymax)/2) * dh
  w_yolo = (xmax-xmin) * dw
  h_yolo = (ymax-ymin) * dh

  return x_center, y_center, w_yolo, h_yolo

In [ ]:
x_center, y_center, w_yolo, h_yolo = convert_to_yolo(img_w, img_h, xmin, ymin, xmax, ymax)

print("x_center:", x_center)
print("y_center:", y_center)
print("w_yolo:", w_yolo)
print("h_yolo:", h_yolo)

* 轉換全部檔案

接著我們需要將全部檔案的格式都進行轉換後，寫成 txt 檔。

要注意的是，在訓練 YOLO 模型時，默認會從資料集圖片路徑中尋找同檔名的標記檔案 (txt 檔)，因此要將圖片和標記檔放在同一個資料夾中。

<img src="https://imgur.com/VuE8kbX.png" width=200>


In [ ]:
train_file = os.path.join(os.getcwd(), 'data_file/train')
valid_file = os.path.join(os.getcwd(), 'data_file/valid')

if not os.path.exists(train_file):
  os.makedirs(train_file)

if not os.path.exists(valid_file):
  os.makedirs(valid_file)

In [ ]:
def create_box_txt(data, images_path, file):
  org_image = ''
  total_img_info = []
  for i in range(len(data)):
    img_info = []
    image = data['image'][i]

    if image != org_image and org_image != '' or i == len(data)-1:
      image_name = org_image.split('.jpg')[0]
      with open(os.path.join(file, image_name + '.txt'), 'w') as f:
        for info in total_img_info:
          f.write('\n'.join(info))
          f.write('\r\n')

      total_img_info = []

    img = cv2.imread(os.path.join(images_path, image))
    img_h, img_w, img_c = img.shape
    

    source = os.path.join(images_path, image)
    destination = os.path.join(file, image)
    shutil.move(source, destination)

    xmin = int(data['xmin'][i])
    ymin = int(data['ymin'][i])
    xmax = int(data['xmax'][i])
    ymax = int(data['ymax'][i])

    x_center, y_center, w_yolo, h_yolo = convert_to_yolo(img_w, img_h, xmin, ymin, xmax, ymax)
    img_info.append(' '.join([str(0),str(x_center),str(y_center),str(w_yolo),str(h_yolo)]))

    total_img_info.append(img_info)

    org_image = image

In [ ]:
create_box_txt(df_data, images_path, train_file)

### 資料集劃分

下一步，我們要來劃分資料集為訓練集 (training set)、驗證集 (validation set)、測試集 (test set)。由於 Kaggle Car Object Detection 資料集已經有區分測試集，因此我們只需要將剛剛轉換完的資料劃分成訓練集與驗證集。

<img src="https://imgur.com/8Tl7N4i.png" width=200>


In [ ]:
def data_process(train_file, valid_file):
  source = os.listdir(train_file)[train_len:]

  for i in source:
    source = os.path.join(train_file, i)
    shutil.move(source, valid_file)

In [ ]:
train_len = int(len(os.listdir(train_file))* 0.8)

data_process(train_file, valid_file)

In [ ]:
print("training data length:", len(os.listdir(train_file)))
print("validate data length:", len(os.listdir(valid_file)))

### 建立 train、valid txt 檔

在這個步驟中，要把 train、valid 資料夾中的**圖片路徑**分別寫入 train.txt、valid.txt，訓練時會根據 txt 裡的路徑去尋找資料。

<img src="https://imgur.com/Kb2FLcj.png" width=700>


In [ ]:
def write_txt_file(data_dir, save_file, txt_name):
  data_list = []
  for i in os.listdir(data_dir):
    if i.endswith('.jpg'):
      data_path = os.path.join(data_dir, i)
      data_list.append(data_path)

  with open(os.path.join(save_file, txt_name), 'a+') as f:
    f.write('\n'.join(data_list))

In [ ]:
save_file = os.path.join(os.getcwd(), 'data_file')

write_txt_file(train_file, save_file, 'train.txt')
write_txt_file(valid_file, save_file, 'valid.txt')

### 建立自定義 yaml 檔

模型在訓練時要讀取的 train、valid txt 檔路徑和總類別數量、類別名稱則會寫在 data 資料夾中的 yaml 檔裡，其中 train/ valid 檔案路徑可寫相對路徑或絕對路徑。

<img src="https://imgur.com/xLGV6tF.png" width=400>


In [ ]:
save_file

In [ ]:
train_txt = os.path.join(save_file, 'train.txt')
valid_txt = os.path.join(save_file, 'valid.txt')

data_dict = {}
data_dict['train'] = train_txt
data_dict['val'] = valid_txt
data_dict['nc'] = 1
data_dict['names'] = ['Car']

with open(os.path.join(os.getcwd(), 'data/car_data.yaml'), 'w') as f:
  yaml.dump(data_dict, f)

### 建立 Config

建立用於訓練的 Config 檔，這邊使用 yolov7-tiny，需修改第二行 nc 為目前訓練資料集的類別數量。

由於 Kaggle Car Object Detection 資料集只有 1 個類別 ('Car')，因此將 nc 改成 1。其他參數也可以根據自己的需求去做更改。

In [ ]:
car_cfg = 'cfg/training/car_yolov7-tiny.yaml'
! cp cfg/training/yolov7-tiny.yaml {car_cfg}
! sed -i '2s/80/1/' {car_cfg}

## 模型訓練 (Training)

提供的超參數設定共有36種，以下介紹幾種常用的超參數

* weights：初始化的模型權重，用於 Transfer Learning

* cfg：模型架構 yaml 檔案

* data：訓練資料檔案

* hyp：超參數設置檔

* epoch：訓練迭代次數

* batch-size：批次大小

* img-size：訓練圖片大小

* rect：是否使用矩形大小的圖片，比如:512x288

* resume：從上一次訓練迭代的地方開始繼續訓練

* device：使用哪個 gpu 或 cpu

* workers：使用多少進程數

* project：訓練結果儲存路徑，默認為 'runs/train'

* name：訓練結果儲存的資料夾名稱，默認為 'exp'

* freeze：凍結哪幾層的網路層

* v5-metric：是否使用 YOLOv5 新制定的 mAP 評估方式 (recall 最大值為 1)

### Train from scratch

* 先來進行 Train from scratch，因此在初始化權重 weights 的部分設定為 ' '

* cfg、data、hyp 路徑設定為剛剛建立的檔案

* epoch 設定為 100、batch size 為 32、圖片大小使用 640x640

* 由於 colab 只有一顆 gpu，因此將 device 設定為 0

* 進程數設定為 1

* 儲存模型的資料夾名稱設定為 yolov7-tiny，因此訓練好的模型會存在 runs/train/yolov7-tiny/weights 資料夾中

In [ ]:
! python train.py --weights '' --cfg cfg/training/car_yolov7-tiny.yaml --data data/car_data.yaml --hyp data/hyp.scratch.tiny.yaml --epochs 100 --batch-size 32 --img 640 640 --device 0 --workers 1 --name yolov7-tiny 

In [ ]:
model_path = os.path.join(os.getcwd(), 'runs/train/yolov7-tiny/weights/best.pth')

In [ ]:
! wget {model_path} -O best.pth

### Transfer Learning

* 接著來使用 Transfer Learning 方法，初始化權重 weights 使用 'yolov7_training.pt'

* cfg、data、hyp 路徑設定為剛剛建立的檔案

* epoch 設定為 100、batch size 為 32、圖片大小使用 640x640

* 由於 colab 只有一顆 gpu，因此將 device 設定為 0

* 進程數設定為 1

* 訓練結果的儲存路徑更改為 runs/transfer_learning

* 儲存模型的資料夾名稱設定為 yolov7-tiny，因此訓練好的模型會存在 runs/transfer_learning/yolov7-tiny/weights 資料夾中

In [ ]:
!wget https://github.com/WongKinYiu/yolov7/releases/download/v0.1/yolov7_training.pt

In [ ]:
! python train.py --weights 'yolov7_training.pt' --cfg cfg/training/car_yolov7-tiny.yaml --data data/car_data.yaml --hyp data/hyp.scratch.tiny.yaml --epochs 100 --batch-size 32 --img 640 640 --device 0 --workers 1 --project runs/transfer_learning --name yolov7-tiny 


## 模型預測 (Predict)

提供的超參數設定共有18種，以下介紹幾種常用的超參數

* weights：要使用的模型權重

* source：要預測的檔案，可以設定為圖片、資料夾路徑或是攝影鏡頭 (設定為0)

* img-size：訓練圖片大小

* conf-thres：object 信心程度的 threshold，默認為 0.25

* iou-thres：用於 NMS 的 IoU threshold，默認為 0.45

* device：使用哪個 gpu 或 cpu

* view-img：是否要顯示畫面

* save-txt；是否要將預測結果儲存至 txt 檔

* save-conf：是否要將預測結果的信心程度儲存至 txt 檔

* project：預測結果儲存路徑，默認為 'runs/detect'

* name：預測結果儲存的資料夾名稱，默認為 'exp'

### Train from scratch

* 接著使用 Train from scratch 所訓練的模型，我們訓練模型儲存路徑在 runs/train/yolov7-tiny/，因此權重設定為在這個路徑下的 weights/best.pt

* 預測圖片選擇在 data_file/test/images/ 路徑下的其中一張圖片




In [ ]:
! python detect.py --weights runs/train/yolov7-tiny/weights/best.pt --source data_file/data/testing_images/vid_5_31560.jpg --project runs/detect

In [ ]:
from PIL import Image
Image.open(os.path.join(os.getcwd(), 'runs/detect/exp/vid_5_31560.jpg'))

### Transfer Learning

* 接著使用 Transfer Learning 所訓練的模型，我們訓練模型儲存路徑在 runs/transfer_learning/yolov7-tiny/，因此權重設定為在這個路徑下的 weights/best.pt

* 預測圖片選擇在 data_file/test/images/ 路徑下的其中一張圖片

* 儲存預測結果的路徑設定為 runs/transfer_learning_detect/

In [ ]:
! python detect.py --weights runs/transfer_learning/yolov7-tiny/weights/best.pt --source data_file/data/testing_images/vid_5_31560.jpg --project runs/transfer_learning_detect

In [ ]:
from PIL import Image
Image.open(os.path.join(os.getcwd(), 'runs/transfer_learning_detect/exp/vid_5_31560.jpg'))

## 模型評估 (Evaluation)

最後來評估模型的 performance

提供的超參數設定共有20種，以下介紹幾種常用的超參數

* weights：要進行評估的模型權重

* data：用於評估的資料檔案

* conf-thres：object 信心程度的 threshold，默認為 0.25

* iou-thres：用於 NMS 的 IoU threshold，默認為 0.45

* task：要使用於 train、validate、test



* project：訓練結果儲存路徑，默認為 'runs/train'

* name：訓練結果儲存的資料夾名稱，默認為 'exp'

* v5-metric：是否使用 YOLOv5 新制定的 mAP 評估方式 (recall 最大值為 1)

### Train from scratch

使用 Train from scratch 所訓練的模型，我們訓練模型儲存路徑在 runs/train/yolov7-tiny/，因此權重設定為在這個路徑下的 weights/best.pt


In [ ]:
! python test.py --weights runs/train/yolov7-tiny/weights/best.pt --data data/car_data.yaml --conf 0.001 --iou 0.65 --img 640 --batch 32 --device 0 --name yolov7-tiny

### Transfer Learning

使用 Transfer Learning 所訓練的模型，我們訓練模型儲存路徑在 runs/transfer_learning/yolov7-tiny/，因此權重設定為在這個路徑下的 weights/best.pt

In [ ]:
! python test.py --weights runs/transfer_learning/yolov7-tiny/weights/best.pt --data data/car_data.yaml --conf 0.001 --iou 0.65 --img 640 --batch 32 --device 0 --name yolov7-tiny